# Ligand ADMET and Potency (Property Prediction)

The ADMET and Potency Challenge of the ASAP Discovery competition take the shape of a property prediction task. Given the SMILES (or, to be more precise, the CXSMILES) of a molecule, you are asked to predict the numerical properties of said molecule. This is a relatively straight-forward application of ML and this notebook will quickly get you up and running!

To begin with, choose one of the two challenges! The code will look the same for both. 

In [2]:
CHALLENGE = "antiviral-admet-2025"  # or: "antiviral-potency-2025"

## Load the competition

Let's first load the competition from Polaris.

Make sure you are logged in! If not, simply run `polaris login` and follow the instructions. 

In [3]:
import polaris as po

competition = po.load_competition(f"asap-discovery/{CHALLENGE}")

2025-01-10 16:32:41.011 | INFO     | polaris._artifact:_validate_version:90 - The version of Polaris that was used to create the artifact (0.2.5) is different from the currently installed version of Polaris (0.10.1.dev1+gabb9b29.d20241218).
2025-01-10 16:32:41.013 | WARNING  | polaris.dataset._base:zarr_root:179 - You're loading data from a remote location. If the dataset is small enough, consider caching the dataset first using CompetitionSpecification.cache() for more performant data access.


As suggested in the logs, we'll cache the dataset. Note that this is not strictly necessary, but it does speed up later steps.

In [4]:
competition.cache()

2025-01-10 16:32:41.488 | INFO     | polaris.dataset._base:_cache_zarr:374 - Copying Zarr archive to /Users/cas.wognum/Library/Caches/polaris/datasets/10cb2cc0-6ab4-43ce-ad9e-8bb95c4c45a9/data.zarr. This may take a while.
2025-01-10 16:32:41.999 | INFO     | polaris.hub.storage:copy_to_destination:177 - Copied 16 (0.07 MiB), skipped 0, of 16 keys. 100.00% completed.


'/Users/cas.wognum/Library/Caches/polaris/datasets/10cb2cc0-6ab4-43ce-ad9e-8bb95c4c45a9'

Let's get the train and test set and take a look at the data structure.

In [5]:
train, test = competition.get_train_test_split()

In [6]:
train[0]

('COC1=CC=CC(Cl)=C1NC(=O)N1CCC[C@H](C(N)=O)C1 |a:16|',
 {'LogD': 0.3, 'MLM': nan, 'MDR1-MDCKII': 2.0, 'KSOL': nan, 'HLM': nan})

In [7]:
test[0]

'CC(C)[C@H]1C2=C(CCN1C(=O)CC1=CN=CC3=CC=CC=C13)SC=C2 |o1:3|'

### Raw data dump

## Build a model
Next, we'll train a simple baseline model using scikit-learn. 

You'll notice that the challenge has multiple targets.

In [8]:
train.target_cols

['LogD', 'MLM', 'MDR1-MDCKII', 'KSOL', 'HLM']

An interesting idea would be to build a multi-task model to leverage shared information across tasks.

For the sake of simplicity, however, we'll simply build a model per target here. 

In [9]:
import datamol as dm
import numpy as np

from sklearn.ensemble import GradientBoostingRegressor

# Prepare the input data. We'll use Datamol to compute the ECFP fingerprints for both the train and test columns.
X_train = np.array([dm.to_fp(dm.to_mol(smi)) for smi in train.X])
X_test = np.array([dm.to_fp(dm.to_mol(smi)) for smi in test.X])

y_pred = {}

# For each of the targets...
for tgt in competition.target_cols:

    # We get the training targets
    # Note that we need to mask out NaNs since the multi-task matrix is sparse.
    y_true = train.y[tgt]
    mask = ~np.isnan(y_true)

    # We'll train a simple baseline model
    model = GradientBoostingRegressor()
    model.fit(X_train[mask], y_true[mask])

    # And then use that to predict the targets for the test set
    y_pred[tgt] = model.predict(X_test)

## Submit your predictions
Submitting your predictions to the competition is simple.

In [10]:
competition.submit_predictions(
    predictions=y_pred,
    prediction_name="my-first-predictions",
    prediction_owner="castest",
    report_url="https://www.example.com", 
    # The below metadata is optional, but recommended.
    github_url="https://github.com/polaris-hub/polaris",
    description="Just testing the Polaris API here!",
    tags=["tutorial"],
    user_attributes={"Framework": "Scikit-learn", "Method": "Gradient Boosting"}
)

✅ SUCCESS: Your competition predictions have been successfully uploaded to the Hub for evaluation.
 


/Users/cas.wognum/micromamba/envs/polaris/lib/python3.12/site-packages/yaspin/core.py:171: UserWarning: color, on_color and attrs are not supported when running in jupyter
  self._color = self._set_color(value) if value else value


For the ASAP competition, we will only evaluate your latest submission. 

The results will only be disclosed after the competition ends.

The End.